In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [2]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/merged.csv')

In [3]:
### Season to test results

N = 2021

In [4]:
data.columns

Index(['season', 'round', 'podium', 'q_delta', 'starting_grid',
       'driver_points_before', 'constructor_points_before',
       'driver_points_from', 'driver_points_per', 'points_percentage',
       ...
       'circuit_id_rodriguez', 'circuit_id_sepang', 'circuit_id_shanghai',
       'circuit_id_silverstone', 'circuit_id_sochi', 'circuit_id_spa',
       'circuit_id_suzuka', 'circuit_id_villeneuve', 'circuit_id_yas_marina',
       'circuit_id_zandvoort'],
      dtype='object', length=116)

In [5]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]
X_train = train.drop(['season', 'round', 'podium', 'driver_points_from'], axis=1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [6]:
X_train

,q_delta,starting_grid,driver_points_before,constructor_points_before,driver_points_per,points_percentage,driver_last_3,constructor_last_3,driver_adrian_sutil,driver_alexander_albon,...,circuit_id_rodriguez,circuit_id_sepang,circuit_id_shanghai,circuit_id_silverstone,circuit_id_sochi,circuit_id_spa,circuit_id_suzuka,circuit_id_villeneuve,circuit_id_yas_marina,circuit_id_zandvoort
0,-0.960826,-1.294941,-0.679459,-0.692428,-0.809546,-1.353452,-0.760054,-0.752773,-0.081289,-0.116955,...,-0.192523,-0.175576,-0.212997,-0.248933,-0.233699,-0.233699,-0.214862,-0.212997,-0.214862,0.0
1,-0.960826,-1.123101,-0.679459,-0.692428,-0.809546,-1.353452,-0.760054,-0.752773,-0.081289,-0.116955,...,-0.192523,-0.175576,-0.212997,-0.248933,-0.233699,-0.233699,-0.214862,-0.212997,-0.214862,0.0
2,-0.872256,-0.092062,-0.679459,-0.692428,-0.809546,-1.353452,-0.760054,-0.752773,-0.081289,-0.116955,...,-0.192523,-0.175576,-0.212997,-0.248933,-0.233699,-0.233699,-0.214862,-0.212997,-0.214862,0.0
3,-0.960826,-0.951261,-0.679459,-0.692428,-0.809546,-1.353452,-0.760054,-0.752773,-0.081289,-0.116955,...,-0.192523,-0.175576,-0.212997,-0.248933,-0.233699,-0.233699,-0.214862,-0.212997,-0.214862,0.0
4,0.722864,0.767138,-0.679459,-0.692428,-0.809546,-1.353452,-0.760054,-0.752773,-0.081289,-0.116955,...,-0.192523,-0.175576,-0.212997,-0.248933,-0.233699,-0.233699,-0.214862,-0.212997,-0.214862,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2737,0.329888,0.595298,-0.620844,-0.632629,-0.762335,0.225815,-0.760054,-0.752773,-0.081289,-0.116955,...,-0.192523,-0.175576,-0.212997,-0.248933,-0.233699,-0.233699,-0.214862,-0.212997,4.654155,0.0
2738,0.413729,1.282658,-0.679459,-0.692428,-0.809546,-1.353452,-0.760054,-0.752773,-0.081289,-0.116955,...,-0.192523,-0.175576,-0.212997,-0.248933,-0.233699,-0.233699,-0.214862,-0.212997,4.654155,0.0
2739,0.164357,1.626338,-0.664805,-0.670003,-0.797743,-0.300712,-0.760054,-0.752773,-0.081289,-0.116955,...,-0.192523,-0.175576,-0.212997,-0.248933,-0.233699,-0.233699,-0.214862,-0.212997,4.654155,0.0
2740,0.297642,1.110818,-0.679459,-0.670003,-0.809546,-1.353452,-0.760054,-0.670118,-0.081289,-0.116955,...,-0.192523,-0.175576,-0.212997,-0.248933,-0.233699,-0.233699,-0.214862,-0.212997,4.654155,0.0


In [7]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(['season', 'round', 'podium', 'driver_points_from'], axis=1)
        y_test = test.podium
        
        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df['predicted'] = prediction_df.proba_1.map(lambda x: 1 if x == prediction_df.proba_1.max() else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df[df.season == N]['round'].unique().max()
    return model_score

In [8]:
comparison_dict ={'model':[],
                  'hidden_layer_sizes': [],
                  'activation': [],
                  'solver': [],
                  'alpha': [],
                  'score': []
                  }

In [9]:
# Neural network

params={'hidden_layer_sizes': [(80,20,40,5), (75,25,50,10)], 
        'activation': ['identity', 'logistic', 'tanh', 'relu'], 
        'solver': ['lbfgs', 'sgd', 'adam'], 
        'alpha': np.logspace(-4,2,20)} 


for hidden_layer_sizes in params['hidden_layer_sizes']:
    for activation in params['activation']:
        for solver in params['solver']:
            for alpha in params['alpha']:
                model_params = (hidden_layer_sizes, activation, solver, alpha)
                model = MLPClassifier(
                    hidden_layer_sizes=hidden_layer_sizes,
                    activation=activation, 
                    solver=solver, 
                    alpha=alpha, 
                    random_state=1)

                model.fit(X_train, y_train)

                model_score = score_classification(model)

                comparison_dict['model'].append('neural_network_classifier')
                comparison_dict['hidden_layer_sizes'].append(hidden_layer_sizes)
                comparison_dict['activation'].append(activation)
                comparison_dict['solver'].append(solver)
                comparison_dict['alpha'].append(alpha)
                comparison_dict['score'].append(model_score)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network

In [ ]:
comparison_df = pd.DataFrame(comparison_dict)

comparison_df.sort_values('score', ascending=False)

,model,hidden_layer_sizes,activation,solver,alpha,score
360,neural_network_classifier,"(75, 25, 50, 10)",tanh,lbfgs,0.000100,0.545455
372,neural_network_classifier,"(75, 25, 50, 10)",tanh,lbfgs,0.615848,0.500000
194,neural_network_classifier,"(80, 20, 40, 5)",relu,lbfgs,2.636651,0.500000
366,neural_network_classifier,"(75, 25, 50, 10)",tanh,lbfgs,0.007848,0.500000
367,neural_network_classifier,"(75, 25, 50, 10)",tanh,lbfgs,0.016238,0.500000
...,...,...,...,...,...,...
329,neural_network_classifier,"(75, 25, 50, 10)",logistic,sgd,0.069519,0.000000
95,neural_network_classifier,"(80, 20, 40, 5)",logistic,sgd,5.455595,0.000000
328,neural_network_classifier,"(75, 25, 50, 10)",logistic,sgd,0.033598,0.000000
327,neural_network_classifier,"(75, 25, 50, 10)",logistic,sgd,0.016238,0.000000


model	hidden_layer_sizes	activation	solver	alpha	score
421	neural_network_classifier	(75, 25, 50, 10)	relu	lbfgs	0.000207	0.5
160	neural_network_classifier	(80, 20, 40, 5)	tanh	adam	0.000100	0.5
162	neural_network_classifier	(80, 20, 40, 5)	tanh	adam	0.000428	0.5
163	neural_network_classifier	(80, 20, 40, 5)	tanh	adam	0.000886	0.5
164	neural_network_classifier	(80, 20, 40, 5)	tanh	adam	0.001833	0.5

In [ ]:
chosen_inputs = comparison_df.query('score >= 0.5')

chosen_inputs.to_csv(path+'parameters/neutralnetworkclassifier.csv')